# Resolución ejercicio 4.6.7
### Análisis y diseño de algoritmos distribuidos en redes
### Andrés Montoro 5.169.779-1



Exercise 4.6.7 

Write the set of rules corresponding to Protocol Iterated_Construction described in Section 4.2.2. Implement and properly test your implementation.


El funcionamiento del protocolo esta explicado enla página 229 del libro de Santoro
Para implementar las iteraciones: un nodo solo comienza una nueva iteración i+1 cuando ha recibido todos los dv de sus vecinos de la iteración i. Si recibe alguno fuera de orden, lo encola y espera que llegue la iteración correspondiente para procesarlo.

Se asumen las restricciones:
- BidirectionalLinks 
- Connectivity
- TotalReliability


In [ ]:
from pydistsim.algorithm.node_algorithm import NodeAlgorithm, StatusValues
from pydistsim.algorithm.node_wrapper import NodeAccess
from pydistsim.message import Message
from pydistsim.restrictions.communication import BidirectionalLinks
from pydistsim.restrictions.reliability import TotalReliability
from pydistsim.restrictions.topological import Connectivity
from pydistsim.restrictions.knowledge import InitialDistinctValues

from pydistsim import NetworkGenerator, Simulation
from pydistsim.logging import set_log_level, LogLevels, enable_logger
from pydistsim.network.behavior import NetworkBehaviorModel
from pydistsim.gui import drawing as draw
%matplotlib inline
from matplotlib import pyplot as plt


## Stages

### Implementacion

In [ ]:
class IteratedConstruction(NodeAlgorithm):
    #Voy a asumir que todos los mensajes son en el formato deseado
    default_params = {
        "communicate" : "Communicate",
    }

    class Status(StatusValues):
        ASLEEP = "ASLEEP"
        CONSTRUCTING = "CONSTRUCTING"
        DONE = "DONE"

    S_init = (Status.ASLEEP)
    S_term = (Status.DONE)

    algorithm_restrictions = (
        BidirectionalLinks,
        Connectivity, 
        TotalReliability, 
        InitialDistinctValues
    )


    def initializer(self):
        self.apply_restrictions()
        for node in self.network.nodes():
            node.status = self.Status.ASLEEP
            node.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=node))

    def initialize(self, node: NodeAccess):
        node.memory["i"] = 0
        node.memory["distance_vector"] = {node.memory["unique_value"] : (None, 0)}
        node.memory["neighbors_dv"] = {}
        node.memory["queue"] = set()


    # Calculo que los mensajes encolados solo pueden ser de la iteracion i+1
    def deque_messages(self, node: NodeAccess):
        messages_to_deque = [m for m in node.memory["queue"] if m.data[1] == node.memory["i"]]
        for message in messages_to_deque:
            node.memory["queue"].remove(message)
            node.push_to_inbox(message)

    def costo(self, node: NodeAccess, neigh_id):
        # Asumo costo 1 entre vecinos
        return 1


    # Precondicion: iter es de la iteracion actual de node
    def process_dvs(self, node: NodeAccess):
        print("Inicializo process_dvs")
        print(node.memory["neighbors_dv"])
        print("Dist vect actual: ", node.memory["distance_vector"])
        w = {}
        for y, Vy in node.memory["neighbors_dv"].items():
            for z, (_, cost_yz) in Vy.items():
                new_wz = self.costo(node, y) + cost_yz
                if z not in w or new_wz < w[z][1]:
                    w[z] = (y, new_wz)
        for z, wz in w.items():
            if z not in node.memory["distance_vector"] or node.memory["distance_vector"][z][1] > wz[1]:
                node.memory["distance_vector"][z] = w[z]
        node.memory["neighbors_dv"] = {neighbor: None for neighbor in node.neighbors()}
        print(node.memory["neighbors_dv"])
        print("Dist vect actual: ", node.memory["distance_vector"])
        print("Finalizo process_dvs")

    @Status.ASLEEP
    def spontaneously(self, node: NodeAccess, message: Message):
        self.initialize(node)
        self.send(
            node,
            data= [node.memory["unique_value"], node.memory["i"], node.memory["distance_vector"]],
            destination= list(node.neighbors()),
            header= self.default_params["communicate"],
        )
        node.status = self.Status.CONSTRUCTING

    @Status.ASLEEP
    def receiving(self, node: NodeAccess, message: Message):
        [neigh_id, _, distance_vector] = message.data
        node.memory["distance_vector"][neigh_id] = (message.source, self.costo(node, neigh_id))
        node.memory["neighbors_dv"][neigh_id] = distance_vector
        self.send(
            node,
            data= [node.memory["unique_value"], node.memory["i"], node.memory["distance_vector"]],
            destination= list(node.neighbors()),
            header= self.default_params["communicate"],
        )
        node.status = self.Status.CONSTRUCTING


    @Status.CONSTRUCTING
    def receiving(self, node: NodeAccess, message: Message):
        [neigh_id, iter, distance_vector] = message.data
        if neigh_id not in node.memory["distance_vector"]:
            node.memory["distance_vector"][neigh_id] = (message.source, self.costo(node, neigh_id))

        print("\n\n")
        print(f"Nodo {node.memory["unique_value"]} - Iteracion {node.memory['i']} - Recibio DV de {neigh_id} en iteracion {iter} : {distance_vector}")
        
        if iter != node.memory["i"]:
            node.memory["queue"].add(message)
            return
        
        node.memory["neighbors_dv"][neigh_id] = distance_vector

        if len(node.memory["neighbors_dv"]) == len(node.neighbors()):
            self.process_dvs(node)

            node.memory["i"] += 1
            node.memory["neighbors_dv"].clear()
            self.send(
                node,
                data= [node.memory["unique_value"], node.memory["i"], node.memory["distance_vector"]],
                destination= list(node.neighbors()),
                header= self.default_params["communicate"],
            )
            self.deque_messages(node)
    
    # @Status.FOLLOWER
    # def default(self, *args, **kwargs):    
    #     pass

### Ejecución


In [ ]:
net_gen = NetworkGenerator(3, directed=False)
net = net_gen.generate_random_network()
sim = Simulation(net, check_restrictions=True)
sim.algorithms = (IteratedConstruction,)
set_log_level(LogLevels.INFO)
enable_logger()

fig = draw.draw_current_state(sim)
fig

Correr la siguiente celda hasta que termine de ejecutarse el algoritmo. Se indica en cada paso:
- El estado del grafo
- Informacion de cada nodo: id, estado, registros en memoria 

In [ ]:
sim.run(1)


### Pruebas

In [ ]:
# ordered_nodes = sorted(net.nodes(), key=lambda n: n.memory["unique_value"])
# assert ordered_nodes[0].status == Stages.Status.LEADER
# assert all(n.status == Stages.Status.FOLLOWER for n in ordered_nodes[1:])
# print("OK")

## Otros

In [ ]:
sim.reset()
plt.close()